In [13]:
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, pooling, Flatten, Dense

#MNIST data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

train_images = train_images.reshape(train_images.shape[0], 32, 32, 3).astype('float32')/255.0

test_images = test_images.reshape(test_images.shape[0], 32, 32, 3).astype('float32')/255.0

train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels) #OneHot encoding

#Model
model = Sequential()
model.add(Conv2D(32, (3,3), padding = 'same', strides = (1,1), activation = 'relu',
                 input_shape = (32,32,3)))
model.add(pooling.MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), padding = 'same', strides = (1,1), activation = 'relu'))
model.add(pooling.MaxPooling2D(pool_size = (2,2)))


model.add(Flatten())
model.add(Dense(10, activation = 'softmax')) #units = 10, activation = softmax
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

#Training & testing
model.fit(train_images, train_labels, epochs = 5, batch_size = 32, verbose = 0)
_, accuracy = model.evaluate(test_images, test_labels)
print('Accuracy : ', accuracy)
model.summary()

170508288/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
Epoch 1/5
1563/1563 [==============================] - 73s 46ms/step - loss: 1.9310 - accuracy: 0.3137
Epoch 2/5
1563/1563 [==============================] - 72s 46ms/step - loss: 1.5711 - accuracy: 0.4442
Epoch 3/5
1563/1563 [==============================] - 72s 46ms/step - loss: 1.4105 - accuracy: 0.5026
Epoch 4/5
1563/1563 [==============================] - 72s 46ms/step - loss: 1.3082 - accuracy: 0.5413
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - loss: 1.2209 - accuracy: 0.5660
Accuracy :  0.5659999847412109
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 3

In [19]:
#CNN with Tensorflow
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Model, layers

batch_size = 128
conv1_filters = 32
conv2_filters = 64
fc1_units = 1024

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train/255. , x_test/255.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size)

class CNN(Model):
  def __init__(self):
    super(CNN, self).__init__()
    #필터사이즈 3by3으로 32개
    self.conv1 = layers.Conv2D(32, kernel_size = 3, activation = tf.nn.relu) 
    self.maxpool1 = layers.MaxPooling2D(2, strides = 2)
    #필터사이즈 3by3으로 64개
    self.conv2 = layers.Conv2D(64, kernel_size = 3, activation = tf.nn.relu) 
    self.maxpool2 = layers.MaxPooling2D(2, strides = 2)

    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(1024)
    self.dropout = layers.Dropout(rate=0.5)
    self.out = layers.Dense(10)

  def call(self, x, is_training = False):
    x = tf.reshape(x, [-1, 28, 28, 1])
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x, training = is_training)
    x = self.out(x)
    return x

CNN_model = CNN()
optimizer = tf.optimizers.Adam(0.01)

In [23]:
def cross_entropy_loss(x,y) :
  y = tf.cast(y, tf.int64)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = x)
  return tf.reduce_mean(loss)

def accuracy(x,y):
  correct = tf.equal(tf.argmax(x,1), tf.cast(y, tf.int64))
  return tf.reduce_mean(tf.cast(correct, tf.float32), axis = -1)

def train_step(x,y) :
  with tf.GradientTape() as g:
    y_pred = CNN_model.call(x)
    loss = cross_entropy_loss(y_pred, y)
    trainable_variables = CNN_model.trainable_variables
  gradients = g.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))

for step, (batch_x, batch_y) in enumerate(train_data.take(300), 1) :
  train_step(batch_x, batch_y)
  if step%10 == 0:
    pred = CNN_model.call(batch_x)
    loss = cross_entropy_loss(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("Step: {},\tAccuracy:{},\tLoss:{}".format(step, acc.numpy().flatten(),
                                                    loss.numpy().flatten()))
    
print('='*30)
print('Model Test')
print('Test Accuracy: ', accuracy(CNN_model(x_test), y_test).numpy().flatten())
print('='*30)

Step: 10,	Accuracy:[0.890625],	Loss:[0.4060443]
Step: 20,	Accuracy:[0.890625],	Loss:[0.405202]
Step: 30,	Accuracy:[0.921875],	Loss:[0.18760344]
Step: 40,	Accuracy:[0.9140625],	Loss:[0.25317726]
Step: 50,	Accuracy:[0.9921875],	Loss:[0.06102899]
Step: 60,	Accuracy:[0.9765625],	Loss:[0.10577503]
Step: 70,	Accuracy:[0.984375],	Loss:[0.0755531]
Step: 80,	Accuracy:[0.921875],	Loss:[0.17373572]
Step: 90,	Accuracy:[0.9453125],	Loss:[0.17547283]
Step: 100,	Accuracy:[0.984375],	Loss:[0.14643331]
Step: 110,	Accuracy:[0.953125],	Loss:[0.13605219]
Step: 120,	Accuracy:[0.9296875],	Loss:[0.23503393]
Step: 130,	Accuracy:[0.984375],	Loss:[0.11176762]
Step: 140,	Accuracy:[0.9765625],	Loss:[0.08834221]
Step: 150,	Accuracy:[0.96875],	Loss:[0.07354632]
Step: 160,	Accuracy:[0.9921875],	Loss:[0.07725519]
Step: 170,	Accuracy:[0.984375],	Loss:[0.06776594]
Step: 180,	Accuracy:[0.984375],	Loss:[0.11114876]
Step: 190,	Accuracy:[0.9921875],	Loss:[0.05385413]
Step: 200,	Accuracy:[0.984375],	Loss:[0.03655626]
Step: 

In [26]:
#CNN with PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from torch.autograd import Variable

train_data = datasets.MNIST(root = 'MNIST_data', train = True, download = True,
                            transform = transforms.ToTensor())
test_data = datasets.MNIST(root = 'MNIST_data', train = True, download = True,
                            transform = transforms.ToTensor())
train_loader = DataLoader(train_data, batch_size = 100, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 100, shuffle = False)

class CNNModel(nn.Module) :
  def __init__(self):
    super(CNNModel, self).__init__()
    self.net = nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
                             torch.nn.ReLU(),
                             torch.nn.MaxPool2d(kernel_size= 2, stride = 2),
                             torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                             torch.nn.ReLU(),
                             torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
                             nn.Dropout(),
                             nn.Flatten(),
                             nn.Linear(7*7*64, 10)
                             )
    
  def forward(self, x):
    return self.net(x)

In [28]:
model = CNNModel()
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
step = 0
loss_list = []
accuracy_list = []

for epoch in range(5):
  for i, (images, labels) in enumerate(train_loader):
    image = Variable(images)
    label = Variable(labels)
    optimizer.zero_grad()
    outputs = model(image)
    loss = cost(outputs, labels)
    loss.backward()
    optimizer.step()
    step+=1

    if step%50 == 0:
      correct = 0
      total = 0
      for images, labels in test_loader:
        image = Variable(images)
        outputs = model(image)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
      accuracy = correct.item()/total
      loss_list.append(loss.data.item())
      accuracy_list.append(accuracy)
      if step%100 == 0:
        print("step:{},\t Accuracy:{},\t Loss:{}".format(step, accuracy, loss))

step:100,	 Accuracy:0.5477333333333333,	 Loss:1.9714809656143188
step:200,	 Accuracy:0.7295,	 Loss:1.0610871315002441
step:300,	 Accuracy:0.7869333333333334,	 Loss:0.7047469615936279
step:400,	 Accuracy:0.8202166666666667,	 Loss:0.5492703318595886
step:500,	 Accuracy:0.8389,	 Loss:0.6393117308616638
step:600,	 Accuracy:0.8503333333333334,	 Loss:0.51429283618927
step:700,	 Accuracy:0.8672,	 Loss:0.46629658341407776
step:800,	 Accuracy:0.8755166666666667,	 Loss:0.43706536293029785
step:900,	 Accuracy:0.8864666666666666,	 Loss:0.3520411550998688
step:1000,	 Accuracy:0.8878166666666667,	 Loss:0.4086596965789795
step:1100,	 Accuracy:0.8989833333333334,	 Loss:0.2904917895793915
step:1200,	 Accuracy:0.90555,	 Loss:0.2476460337638855
step:1300,	 Accuracy:0.9090166666666667,	 Loss:0.40266337990760803
step:1400,	 Accuracy:0.91405,	 Loss:0.19300086796283722
step:1500,	 Accuracy:0.9164666666666667,	 Loss:0.29761767387390137
step:1600,	 Accuracy:0.9205166666666666,	 Loss:0.25141769647598267
step:17

In [29]:
#Keras
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, pooling, Flatten, Dense

#MNIST type
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')/255.0
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')/255.0

train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels) #OneHot encoding

#Model
model = Sequential()
model.add(Conv2D(32, (3,3), padding = 'same', strides = (1,1), activation = 'relu',
                 input_shape = (28,28,1)))
print(model.output_shape)

model.add(pooling.MaxPooling2D(pool_size = (2,2)))
print(model.output_shape)

model.add(Conv2D(64, (3,3), padding = 'same', strides = (1,1), activation = 'relu'))
print(model.output_shape)

model.add(pooling.MaxPooling2D(pool_size = (2,2)))
print(model.output_shape)

model.add(Flatten())
model.add(Dense(10, activation = 'softmax')) #units = 10, activation = softmax
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

#Training & testing
model.fit(train_images, train_labels, epochs = 5, batch_size = 32, verbose = 0)
_, accuracy = model.evaluate(test_images, test_labels)
print('Accuracy : ', accuracy)
model.summary()

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(None, 28, 28, 32)
(None, 14, 14, 32)
(None, 14, 14, 64)
(None, 7, 7, 64)
313/313 [==============================] - 3s 11ms/step - loss: 0.0639 - accuracy: 0.9799
Accuracy :  0.9799000024795532
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
       